In [5]:
from nltk import ne_chunk, pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
#from nltk.stem import WordNetLemmatizer
#wn_lem = WordNetLemmatizer() # wn_lem.lemmatize()


sent = "WASHINGTON-- In the wake of a string of abuses by New York police officers in the 1990s, she, Loretta E. Lynch, the top federal prosecutor in Brooklyn, spoke forcefully about the pain of a broken trust that African-Americans felt and said the responsibility for repairing generations of miscommunication and mistrust fell to law enforcement she."

"""
    Input: (data, word_range, which_type)
        data - one sentence
        word_range - a nested list of clusters with their ranges (ie. [[[0,0],[0,19]],[[19,19],[59,59]]])
        which_type - one of ("name", "pro", "both"/nothing)
        
    return case: 
        1. if which_type == "name" and the function returns True >> THERE IS NAME LINK >> REMOVE
        2. if which_type == "pro" and function returns True >> ALL LINKS ARE PRONOUN >> REMOVE
        3. if which_type not given and function returns Ture >> 
                (there is either or both NAME LINK OR ALL THE LINKS ARE PRONOUNS) >> REMOVE
        4. if the function returns False >> NO NAME LINK and ALL LINKS ARE NOT PRONOUNS >> KEEP
                   
"""

def check_remove(data, word_range, which_type="both"):
    
    pro_lst = ["he", "she", "him", "her", "his", "hers", "himself", "herself"]
    name_lst = []
    result = []
    tok = word_tokenize(data)

    # Check all the instances of human names in the sentence and build "name_lst"
    for sent_chunk in ne_chunk(pos_tag(word_tokenize(data))):
        if hasattr(sent_chunk, 'label'):
            if (sent_chunk.label() == "PERSON"):
                name_lst.append(' '.join(c[0] for c in sent_chunk))

    for cluster in word_range:
        if (which_type == "name"): # check if the cluster has name link
            result.append(any([(' '.join(w for w in tok[c[0]:c[1]+1])) in name_lst for c in cluster]))
            
        elif (which_type == "pro"): # check if the cluster has only pronoun links
            result.append(all([((c[0] == c[1]) and (tok[c[0]]).lower() in pro_lst) for c in cluster]))
            
        else: # check both conditions at the same time
            result.append(any([any([(' '.join(w for w in tok[c[0]:c[1]+1])) in name_lst for c in cluster]),
                   all([((c[0] == c[1]) and (tok[c[0]]).lower() in pro_lst) for c in cluster])]))
    
    return any(result)


In [10]:
check_remove(sent, [[[0,0],[0,19]],[[19,19],[59,59]]], "name")

check_remove(sent, [[[21,23],[19,19]],[[19,19],[59,59]]], "pro")
#check_remove(sent, [[[19,19],[59,59]]], "pro")
check_remove(sent, [[[21,23]]])

True